# Import modules

In [3]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
# pd.set_option('display.max_colwidth', -1)

import warnings
warnings.filterwarnings('ignore')

import os
import sys
sys.path.insert(0, '../')
from scpts import alziras

import basedosdados as bd

out_path = '../data/analise/mais/alziras/output/'
in_path  = '../data/analise/mais/alziras/input/'

### Download do basedosdados

In [4]:
# ##-----------------------------###
# q_municipios = """
# SELECT id_municipio,id_municipio_tse,municipio,regiao
# FROM `basedosdados.br_bd_diretorios_brasil.municipio`
# """
# municipios = bd.read_sql(q_municipios, billing_project_id="basedosdados-dev")
# municipios.to_parquet(f"{in_path}municipios.parquet")

# ## Descomentar para rodar

q_resultado_candidatos = """
SELECT * FROM `basedosdados.br_tse_eleicoes.resultados_candidato` 
where (ano=2016 or ano=2020)
"""
re_secao = bd.read_sql(q_resultado_candidatos, billing_project_id="basedosdados-dev")
re_secao.to_parquet(f"{in_path}resultado_candidatos.parquet")

# ###-----------------------------###
# q_candidatos = """
# SELECT * FROM `basedosdados.br_tse_eleicoes.candidatos`
# where (ano=2016 or ano=2020) 
# """
# candidatos_capitais = bd.read_sql(q_candidatos, billing_project_id="basedosdados-dev")
# candidatos_capitais.to_parquet(f"{in_path}candidatos.parquet")

# ##-----------------------------###
# q_municipios = """
# SELECT id_municipio,id_municipio_tse,municipio,regiao
# FROM `basedosdados.br_bd_diretorios_brasil.municipio`
# """
# municipios = bd.read_sql(q_municipios, billing_project_id="basedosdados-dev")
# municipios.to_parquet(f"{in_path}municipios.parquet")

# ##-----------------------------###
# q_municipios_pib = """
# with pib_municipios as (
# select id_municipio,ano,PIB
# from `basedosdados.br_ibge_pib.municipios`
# where(ano=2016 or ano=2018)
# ),

# municipios_pop as (
# select *
# from `basedosdados.br_ibge_populacao.municipios`
# where(ano=2016 or ano=2018)
# )

# select * from pib_municipios
# join municipios_pop on (municipios_pop.id_municipio = pib_municipios.id_municipio) and (municipios_pop.ano = pib_municipios.ano)
# """
# pib_municipios = bd.read_sql(q_municipios_pib, billing_project_id="basedosdados-dev").drop(["id_municipio_1", "ano_1"], 1)
# pib_municipios.to_parquet(f"{in_path}municipios_pib.parquet")

# q_bens_candidato = """
# SELECT * FROM `basedosdados.br_tse_eleicoes.bens_candidato`
# where (ano=2016 or ano=2020)
# """
# besn = bd.read_sql(q_bens_candidato, billing_project_id="basedosdados-dev")
# besn.to_parquet(f"{in_path}bens_candidato.parquet")

# q_receita_candidatos = """
# SELECT * FROM `basedosdados-dev.br_tse_eleicoes.receitas_candidato`
# where (ano=2016 or ano=2020)
# """
# receita = bd.read_sql(q_receita_candidatos, billing_project_id="basedosdados-dev")
# receita.to_parquet(f"{in_path}receita_candidatos.parquet")

# q_despesa_candidatos = """
# SELECT * FROM `basedosdados-dev.br_tse_eleicoes.despesas_candidato`
# where (ano=2016 or ano=2020)
# """
# despesa = bd.read_sql(q_despesa_candidatos, billing_project_id="basedosdados-dev")
# despesa.to_parquet(f"{in_path}despesa_candidatos.parquet")


Downloading: 100%|██████████| 965432/965432 [00:13<00:00, 69390.11rows/s] 


# Prefeitas e Vereadoras

### Import Data

In [5]:
cadidato_resultado = pd.read_parquet(f"{in_path}resultado_candidatos.parquet")
candidatos = pd.read_parquet(f"{in_path}candidatos.parquet")
municipios = pd.read_parquet(f"{in_path}/municipios.parquet")
pib_municipios = pd.read_parquet(f"{in_path}municipios_pib.parquet")

In [6]:
cadidato_resultado['id_municipio_tse'] = cadidato_resultado['id_municipio_tse'].astype('str')
candidatos['id_municipio_tse'] = candidatos['id_municipio_tse'].astype('str')
pib_municipios['id_municipio'] = pib_municipios['id_municipio'].astype('str')

In [7]:
final_table = alziras.get_final_table(
    cadidato_resultado, candidatos, municipios, pib_municipios
)

In [8]:
# mask = final_table['municipio']=='Manaus'
# final_table[mask]

### Gera Tabelas
-- --

**Definições**
 * Utilizei o PIB de 2018 como a versão mais recente para o calculo do PIB percapita de 2020
 * Utilizei Negros como sendo pretos + pardos
-- --
**Duvidas**
 * 0 a 20 mil - 20 a 30mil ?? não seria 0 a 20 mil - 21 a 30 mil para não repitir o 20 mil. Mesmo vale para faixa etaria
 * Mulheres são todas as mulheres ou todas menos mulheres negras?
 * Proporção de negros em relção ao total de negros daquele atributo ou proporção de negros em relação ao total do genero?
 -- --

**Sugestões**
 * Quebrar por resuldado??
 * Possibilidade de gerar mais atributos de agregação utilizando id_municipio
-- --


### Geral

In [9]:
## padroniza eleitos
final_table["resultado"] = (
    final_table["resultado"]
    .str.replace("eleito por qp", "eleito")
    .str.replace("eleito por media", "eleito")
)

## somente eleitos
mask = final_table["resultado"].isin(["eleito"])
final_table = final_table[mask]

## genero declarado
mask = final_table["genero"].notnull()
final_table = final_table[mask]

## raca declarada
mask = final_table["raca"].notnull()
final_table = final_table[mask]

In [10]:
### Genero
group_cols = ["ano", "cargo", "genero", "raca"]  # + ['sigla_uf']
select_cols = ["contagem", "votos"]
atributo = None

df_atributo, df_total = alziras.group_genero_raca(
    final_table, group_cols, select_cols, atributo
)

# df_atributo.to_csv(f"{out_path}contagem_e_votos.csv", index=False)

### Atributos

In [11]:
### Genero
group_cols = ["ano", "cargo", "genero", "raca"]  # + ['sigla_uf']
select_cols = ["contagem", "votos"]
atributos = [
    "instrucao",
    "faixa_etaria",
    "ocupacao",
    "estado_civil",
    "sigla_uf",
    "regiao",
    "porte_municipal",
    "decil",
    "sigla_partido",
]

for atributo in atributos:

    df_atributo, df_total = alziras.group_genero_raca(
        final_table, group_cols, select_cols, atributo, out_path, save=True
    )
    print(atributo)
    print("quantidade de linhas = ", df_atributo.shape[0])
    print(f"quantidade de {atributo} = ", len(df_atributo[atributo].unique()), "\n")


instrucao
quantidade de linhas =  280
quantidade de instrucao =  7 

faixa_etaria
quantidade de linhas =  200
quantidade de faixa_etaria =  5 

ocupacao
quantidade de linhas =  9240
quantidade de ocupacao =  231 

estado_civil
quantidade de linhas =  160
quantidade de estado_civil =  4 

sigla_uf
quantidade de linhas =  1040
quantidade de sigla_uf =  26 

regiao
quantidade de linhas =  200
quantidade de regiao =  5 

porte_municipal
quantidade de linhas =  200
quantidade de porte_municipal =  5 

decil
quantidade de linhas =  400
quantidade de decil =  10 

sigla_partido
quantidade de linhas =  1320
quantidade de sigla_partido =  33 



# Financeiro

### Import Data

In [12]:
##### load despesa
load_cols_despesa = ["ano", "sequencial_candidato", "valor_despesa"]

despesa = pd.read_parquet(
    f"{in_path}despesa_candidatos.parquet", columns=load_cols_despesa
)

##### load receita
load_cols_receita = [
    "ano",
    "sequencial_candidato",
    "sigla_uf",
    "origem_receita",
    "fonte_receita",
    "valor_receita",
]

receita = pd.read_parquet(
    f"{in_path}receita_candidatos.parquet", columns=load_cols_receita
)

### remove valores preenchidos errado
mask = (receita["sigla_uf"] == "AM") & (receita["valor_receita"] > 10000000)
receita = receita[np.logical_not(mask)]


##### load bens
load_cols_bens = ["ano", "sequencial_candidato", "valor_item"]

bens = pd.read_parquet(f"{in_path}bens_candidato.parquet", columns=load_cols_bens)

In [13]:
cadidato_resultado = pd.read_parquet(f"{in_path}resultado_candidatos.parquet")
candidatos = pd.read_parquet(f"{in_path}candidatos.parquet")

municipios = pd.read_parquet(f"{in_path}/municipios.parquet")
pib_municipios = pd.read_parquet(f"{in_path}municipios_pib.parquet")

cadidato_resultado['id_municipio_tse'] = cadidato_resultado['id_municipio_tse'].astype('str')
candidatos['id_municipio_tse'] = candidatos['id_municipio_tse'].astype('str')
pib_municipios['id_municipio'] = pib_municipios['id_municipio'].astype('str')

In [14]:
final_table = alziras.get_final_table(
    cadidato_resultado, candidatos, municipios, pib_municipios
)

## padroniza eleitos
final_table["resultado"] = (
    final_table["resultado"]
    .str.replace("eleito por media", "eleito")
    .str.replace("eleito por qp", "eleito")
)
## raca declarada
mask = final_table["raca"].notnull()
final_table = final_table[mask]

### Manipulate Tables

In [15]:
from scpts import alziras

receita_final, despesa_final, bens_final = alziras.get_financeiro_bens(
    final_table, bens, receita, despesa
)

### Gera Tabelas
-- --

**Definições**
 * Utilizei Negros como sendo pretos + pardos
 * Eleitos = eleito por media + eleito por qp
-- --
**Duvidas**
 * Gasto?
 * O que fazer com suplentes?
 -- --

### Receita total

In [16]:
group_cols = ["ano", "cargo", "genero", "raca", "resultado"]  # + ['sigla_uf']
select_cols = ["contagem", "valor_receita"]
atributo = None

receita_total, df_total = alziras.group_receita(
    receita_final, group_cols, select_cols, atributo
)
receita_total = receita_total[receita_total["resultado"] != "suplente"]

receita_total.to_csv(f"{out_path}receita_total.csv", index=False)


### Receita por atributo

In [17]:
group_cols = ["ano", "cargo", "genero", "raca", "resultado"]  # + ['sigla_uf']
select_cols = ["contagem", "valor_receita"]
atributo = "origem_receita"

receita_origem, df_total = alziras.group_receita(
    receita_final, group_cols, select_cols, atributo
)
receita_origem = receita_origem[receita_origem["resultado"] != "suplente"]
receita_origem.to_csv(f"{out_path}receita_por_origem.csv", index=False)

mask_proprio = receita_origem["origem_receita"] == "recursos proprios"
receita_origem[mask_proprio].to_csv(
    f"{out_path}receita_por_origem_proprio.csv", index=False
)

mask_pf = receita_origem["origem_receita"] == "recursos de pessoas fisicas"
receita_origem[mask_pf].to_csv(f"{out_path}receita_por_origem_pf.csv", index=False)

mask_crowdfunding = (
    receita_origem["origem_receita"] == "recursos de financiamento coletivo"
)
receita_origem[mask_crowdfunding].to_csv(
    f"{out_path}receita_por_origem_crowdfunding.csv", index=False
)

In [18]:
### Receita por fonte
group_cols = ["ano", "cargo", "genero", "raca", "resultado"]  # + ['sigla_uf']
select_cols = ["contagem", "valor_receita"]
atributo = "fonte_receita"

receita_fonte, df_total = alziras.group_receita(
    receita_final, group_cols, select_cols, atributo
)
receita_fonte = receita_fonte[receita_fonte["resultado"] != "suplente"]

receita_fonte.to_csv(f"{out_path}receita_por_fonte.csv", index=False)

### Patrimonio Total

In [19]:
group_cols = ["ano", "cargo", "genero", "raca", "resultado"]  # + ['sigla_uf']
select_cols = ["contagem", "patrimonio"]
atributo = None

patrimonio, df_total = alziras.group_receita(
    bens_final, group_cols, select_cols, atributo
)
patrimonio = patrimonio[patrimonio["resultado"] != "suplente"]

rename_cols = {
    "valor_receita": "patrimonio",
    "valor_receita_total": "patrimonio_total",
    "valor_receita_proporcional": "patrimonio_proporcional",
}
patrimonio = patrimonio.rename(columns=rename_cols)
patrimonio = patrimonio.drop(["valor_receita_per_capta"], 1)

patrimonio.to_csv(f"{out_path}patrimonio_total.csv", index=False)

### Despesa Total

In [20]:
group_cols = ["ano", "cargo", "genero", "raca", "resultado"]  # + ['sigla_uf']
select_cols = ["contagem", "valor_despesa"]
atributo = None

despesa_total, df_total = alziras.group_receita(
    despesa_final, group_cols, select_cols, atributo
)
despesa_total = despesa_total[despesa_total["resultado"] != "suplente"]

rename_cols = {
    "valor_receita": "valor_despesa",
    "valor_receita_total": "valor_despesa_total",
    "valor_receita_proporcional": "valor_despesa_proporcional",
    "valor_receita_per_capta": "valor_despesa_per_capta",
}
despesa_total = despesa_total.rename(columns=rename_cols)

despesa_total.to_csv(f"{out_path}despesa_total.csv", index=False)

# Demanda Adicional

In [21]:
cadidato_resultado = pd.read_parquet(f"{in_path}resultado_candidatos.parquet")

candidatos = pd.read_parquet(f"{in_path}candidatos.parquet")

municipios = pd.read_parquet(f"{in_path}/municipios.parquet")
pib_municipios = pd.read_parquet(f"{in_path}municipios_pib.parquet")

### load receita
load_cols = [
    "ano",
    "sigla_uf",
    "id_municipio_tse",
    "turno",
    "sequencial_candidato",
    "numero_candidato",
    "sigla_partido",
    "cargo",
    "origem_receita",
    "fonte_receita",
    "valor_receita",
]

receita = pd.read_parquet(f"{in_path}receita_candidatos.parquet", columns=load_cols)
mask = (receita["sigla_uf"] == "AM") & (receita["valor_receita"] > 10000000)
receita = receita[np.logical_not(mask)]

In [22]:
receita['id_municipio_tse'] = receita['id_municipio_tse'].astype(str)
cadidato_resultado['id_municipio_tse'] = cadidato_resultado['id_municipio_tse'].astype('str')
candidatos['id_municipio_tse'] = candidatos['id_municipio_tse'].astype('str')
pib_municipios['id_municipio'] = pib_municipios['id_municipio'].astype('str')

In [23]:
tabela_final = alziras.demanda_adicional(
    cadidato_resultado, 
    candidatos, 
    municipios, 
    pib_municipios, 
    receita
)

In [24]:
tabela_final[['ano','valor_receita']].groupby(by='ano').sum()

,valor_receita
ano,
2016,2.995349e+09
2020,3.185731e+09


In [25]:
receita[['ano','valor_receita']].groupby(by='ano').sum()

,valor_receita
ano,
2016,3.030411e+09
2020,3.405879e+09


In [26]:
mask_prefeito = tabela_final['cargo']=='prefeito'
tabela_final[mask_prefeito].to_csv(f'{out_path}prefeitos_completo.csv', index=False)
tabela_final[np.logical_not(mask_prefeito)].to_csv(f'{out_path}vereadores_completo.csv', index=False)

### Bug

In [27]:
vereadoras = pd.read_csv('../data/analise/mais/alziras/output/vereadores_completo.csv')
entrega_vereadores = pd.read_csv('../data/analise/mais/alziras/output/vereadores_completo_entrega.csv')
municipios = pd.read_parquet('../data/analise/mais/alziras/input/municipios.parquet')

vereadoras['id_municipio'] = vereadoras['id_municipio'].astype('str')
municipios['id_municipio'] = municipios['id_municipio'].astype('str')
entrega_vereadores['id_municipio'] = entrega_vereadores['id_municipio'].astype('str')



In [28]:
entrega_vereadores_2020 = vereadoras[vereadoras['ano']==2020]

In [34]:
mask = ~municipios['id_municipio'].isin(entrega_vereadores_2020['id_municipio'].unique())

len(municipios[mask])

53

In [30]:
mask = ~municipios['id_municipio'].isin(entrega_vereadores_2020['id_municipio'].unique())


In [31]:
def get_bug(df):
    mask = (df['id_municipio_tse']=='2550') & (df['ano']==2020)
    return df[mask]

In [32]:
# cadidato_resultado = pd.read_parquet(f"{in_path}resultado_candidatos.parquet")
# candidatos = pd.read_parquet(f"{in_path}candidatos.parquet")
# municipios = pd.read_parquet(f"{in_path}/municipios.parquet")
# pib_municipios = pd.read_parquet(f"{in_path}municipios_pib.parquet")

In [33]:
get_bug(cadidato_resultado)

,ano,tipo_eleicao,turno,sigla_uf,id_municipio_tse,numero_candidato,id_candidato_bd,cargo,sigla_partido,resultado,votos
964826,2020,eleicao ordinaria,2,AM,2550,19,12051.0,prefeito,PODE,nao eleito,443747
964827,2020,eleicao ordinaria,2,AM,2550,70,1124558.0,prefeito,AVANTE,eleito,466970


In [ ]:
len(vereadoras['id_municipio'].unique().tolist())